In [ ]:
# TODO: Mejoras para este ejercicio:
# - Asegurar que los datos sean consistentes: si hay valores erróneos o nulos, establecer un random_state para garantizar reproducibilidad
# - Implementar pipelines para estructurar mejor el flujo de preprocesamiento y modelado, evitando la aplicación manual de cada paso
# - Optimizar el modelo ajustando hiperparámetros con técnicas como GridSearchCV o RandomizedSearchCV
# - Utilizar Regex para validaciones: códigos postales, teléfonos, emails, etc.
# - Crear variables derivadas como precio por metro cuadrado (precio_m2 = precio / superficie)
# - Geolocalización: obtener coordenadas con OpenStreetMap a partir de direcciones o códigos postales y utilizarlas para análisis espaciales
# - Visualizar las viviendas en un mapa interactivo con Folium o Plotly Express para identificar patrones geográficos en los precios
# - Clusterización de zonas con K-Means o DBSCAN para detectar patrones de precios por ubicación y segmentar mejor los inmuebles
# - Evitar data leakage: Dividir los datos en train/test antes de hacer encoding, eliminar outliers o escalar,
#   asegurando que las transformaciones se ajusten sólo con el conjunto de entrenamiento y luego se apliquen en test
# - Subir el proyecto final a Kaggle

## **Funciones Helper**
Aquí se encuentran las funciones auxiliares utilizadas en este notebook. Estas funciones permiten realizar tareas específicas de manera más ordenada y modular.

In [ ]:
def clean_floor_column(df):
    """
    Limpia la columna 'floor' asegurando consistencia en los valores.
    - Convierte valores ordinales como '1st', '2nd', '3rd' en números.
    - Sustituye el valor 'ground' por 0.
    - Convierte en NaN otros valores como 'floor'.
    - Maneja valores extremadamente altos o negativos.
    - Convierte a tipo Int16 para optimizar memoria.
    """

    # Mostrar valores únicos antes del saneamiento
    print("\nValores únicos en 'floor' antes del saneamiento:")
    print(df["floor"].unique())

    # Diccionario de conversión de valores ordinales (1st, 2nd, etc.)
    ordinal_mapping = {f"{i}th": i for i in range(1, 101)}
    ordinal_mapping.update({"ground": 0, "1st": 1, "2nd": 2, "3rd": 3})

    # Convertir valores de la columna
    cleaned_floors = []
    for value in df["floor"]:
        if pd.isna(value):
            cleaned_floors.append(pd.NA)  # Mantener valores nulos
            continue

        value = str(value).lower().strip()

        # Intentar convertir directamente si es un número
        try:
            num_value = int(value.replace(",", ""))  # Manejo de valores con comas como "1,200"
            if 0 <= num_value <= 100:  # Limitar a un rango razonable de pisos
                cleaned_floors.append(num_value)
            else:
                cleaned_floors.append(pd.NA)  # Si el número es absurdo, lo dejamos como NA
        except ValueError:
            cleaned_floors.append(ordinal_mapping.get(value, pd.NA))  # Mapear valores ordinales o NA si no se reconoce

    # Asignar la columna limpia al DataFrame
    df["floor"] = cleaned_floors

    # Convertir a Int16 para optimizar memoria, manteniendo NaN con pd.NA
    df["floor"] = df["floor"].astype("Int16") # TODO Hacer esto en la conversión

    # Mostrar valores únicos después del saneamiento
    print("\nValores únicos en 'floor' después del saneamiento:")
    print(df["floor"].unique())

    return df



def is_within_madrid(lat, lng):
    """
    Verifica si una ubicación dada por latitud y longitud está dentro de un radio determinado
    desde el centro de Madrid.

    Parameters:
    ----------
    lat: float
        Latitud de la ubicación a verificar.
    lng: float
        Longitud de la ubicación a verificar.

    Returns:
    -------
    bool
        True si la ubicación está dentro del radio permitido, False en caso contrario.

    Notes:
    ------
    - Se toma como referencia el centro de Madrid con coordenadas (40.4168, -3.7038).
    - Se define un umbral de distancia máxima de 60 km para considerar una ubicación válida.
    - Se usa la función `geodesic` de geopy para calcular la distancia en kilómetros.
    - Si la latitud o longitud son nulas, la función retorna False para evitar errores.
    """

    madrid_center = (40.4168, -3.7038)
    radius_km = 60

    if pd.isna(lat) or pd.isna(lng):
        return False

    distance = geodesic((lat, lng), madrid_center).km  # Calcula la distancia al centro de Madrid
    return distance <= radius_km  # Devuelve True si está dentro del radio, False si está fuera

## 1 - Carga de datos y revisión de la estructura del dataset

En este apartado se realiza una **primera exploración del dataset** para comprender su estructura, tipo de variables y posibles relaciones entre ellas. Este paso es fundamental para la correcta preparación de los datos antes del modelado.

### **1.1 - Importación de librerías y configuración**
Se importan las librerías esenciales para el análisis de datos, la visualización y el modelado, incluyendo **Pandas, NumPy, Matplotlib, Seaborn, SciPy y Scikit-Learn**. Además, se configuran algunos parámetros globales de visualización para mejorar la legibilidad de los gráficos.

In [ ]:
# Cargar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
import tensorflow as tf
import folium
from geopy.distance import geodesic
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import folium
from folium.plugins import MarkerCluster

# Configuraciones
%matplotlib inline
sns.set_theme(style="whitegrid", palette="viridis", font_scale=1.1)

### **1.2 - Carga del dataset**
Se carga el conjunto de datos en un **DataFrame de Pandas** desde un archivo CSV. Se incluye una referencia a la fuente del dataset.

In [ ]:
# Cargar el dataset local con Pandas
df = pd.read_csv("scripts/madrid_rent_with_coordinates.csv")

# Ref. https://www.kaggle.com/datasets/mapecode/madrid-province-rent-data

### **1.3 - Inspección de la estructura del dataset**
En esta fase se inspecciona la estructura general del dataset para entender su contenido y formato:

- **Visualización de las primeras y últimas filas** para detectar posibles errores en la carga de los datos.
- **Información general del dataset** (`df.info()`), que muestra el número de registros, tipos de datos y valores nulos.
- **Número total de filas y columnas** (`df.shape`).
- **Identificación de columnas numéricas y categóricas**, que ayudará en el preprocesamiento.
- **Recuento de valores únicos en variables categóricas**, útil para evaluar su diversidad.
- **Visualización de los valores únicos en algunas columnas categóricas clave**, como `type`, `floor`, `orientation`, `district` y `subdistrict`, con el objetivo de analizar su distribución y detectar posibles inconsistencias.
- **Revisión del número de valores nulos por columna** (`df.isnull().sum()`), lo que permitirá identificar qué variables requieren imputación o eliminación en la etapa de preprocesamiento.
- **Resumen estadístico de las variables numéricas**, para analizar su distribución, detectar valores atípicos y entender la escala de los datos.

In [ ]:
# Mostrar las primeras filas para una vista inicial del dataset
print("Primeras filas del dataset:")
display(df.head())

In [ ]:
# Mostrar las últimas filas para identificar posibles problemas en la carga de datos
print("Últimas filas del dataset:")
display(df.tail())

In [ ]:
# Obtener información general sobre el dataset, incluyendo tipos de datos y valores nulos
print("Información general del dataset:")
display(df.info())

In [ ]:
# Mostrar el número de filas y columnas en el dataset
print("Número de filas y columnas en el dataset:")
print(df.shape)

In [ ]:
# Identificar las columnas numéricas y categóricas
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
# Mostrar las columnas numéricas identificadas
print("Columnas numéricas:")
print(numerical_columns)

In [ ]:
# Mostrar las columnas categóricas identificadas
print("Columnas categóricas:")
print(categorical_columns)

In [ ]:
# Contar valores únicos en variables categóricas
print("Valores únicos en variables categóricas:")
df[categorical_columns].nunique()

In [ ]:
# Mostrar valores únicos de algunas columnas categóricas para revisar su diversidad
relevant_categorical_columns = ['type', 'floor', 'orientation', 'district', 'subdistrict']

print("Valores únicos en algunas variables categóricas relevantes:")
for col in relevant_categorical_columns:
    print(f"\n{col}: {df[col].nunique()} valores únicos")
    print(df[col].unique()[:10])

In [ ]:
# Mostrar el número de valores nulos por columna
print(df.isnull().sum())

In [ ]:
# Describir estadísticamente las variables numéricas para analizar su distribución y posibles valores atípicos
print("Resumen estadístico de las variables numéricas:")
display(df.describe())

### **1.4 - Comprobación de relaciones potenciales**
Para evaluar la viabilidad del modelado, se analizan las correlaciones entre variables numéricas:

- Se genera una **matriz de correlación** (`df.corr()`) para medir la relación entre variables.
- Se identifican **las variables con mayor impacto en `price`** basándose en la correlación:
  - `floor_built` (0.70)
  - `floor_area` (0.72)
  - `bedrooms` (0.51)
  - `bathrooms` (0.69)
- Se confirma la pertinencia de `balcony` como una **variable categórica de clasificación**, revisando la distribución de sus valores.

In [ ]:
# Se genera la matriz de correlación para analizar la relación entre las variables numéricas
print("Matriz de correlación entre variables numéricas:")
correlation_matrix = df[numerical_columns].corr()
display(correlation_matrix)

In [ ]:
# Identificar relaciones relevantes para modelado

# Basándonos en la matriz de correlación, seleccionamos las variables con mayor impacto en 'price'
# Según el análisis previo, las variables con correlación más fuerte con 'price' son:
# - floor_built (0.70)
# - floor_area (0.72)
# - bedrooms (0.51)
# - bathrooms (0.69)
# Otras variables tienen correlaciones insignificantes (<0.01) y no se consideran para modelado

key_relationships = ['price', 'floor_built', 'floor_area', 'bedrooms', 'bathrooms']
print("Relaciones clave para la predicción de price:")
display(df[key_relationships].corr())

In [ ]:
# Se selecciona balcony como variable de clasificación
# Se analiza la frecuencia de los valores presentes en balcony para entender su distribución
print("Frecuencia de valores en la variable balcony:")
df['balcony'].value_counts()

## **2 - Limpieza y validación de los datos**

Este apartado se centra en la **depuración del dataset** para garantizar que los datos sean consistentes, relevantes y adecuados para el modelado. Se eliminan columnas innecesarias, se manejan valores duplicados y se corrigen inconsistencias en los valores.

### **2.1 - Eliminación de columnas irrelevantes**
Se eliminan columnas que no aportan información útil para el análisis de precios y balcones. Estas incluyen identificadores únicos (`web_id`), enlaces (`url`), información redundante (`title`), y metadatos sin valor predictivo (`professional_name`, `last_update`, `location`).

In [ ]:
# Se eliminan columnas que no aportan información útil para el análisis de precios y balcones
columns_to_drop = [
    'web_id',               # Identificador único para cada anuncio, no aporta valor analítico
    'url',                  # Enlace a la página del anuncio, irrelevante para el modelado
    'title',                # Contiene información redundante, ya que 'type' y 'location' están en otras columnas
    'professional_name',    # Nombre de la agencia inmobiliaria o propietario, sin impacto en la predicción
    'last_update',          # Indica la fecha de última actualización del anuncio, pero sin un formato uniforme y sin valor predictivo
    'location'              # Contiene el nombre de la calle y en algunos casos el número, pero ya se tiene 'district' y 'subdistrict'
]

# Eliminamos las columnas del dataframe
df.drop(columns=columns_to_drop, inplace=True)

# Mostramos el nuevo número de columnas para verificar la eliminación
print("Columnas eliminadas:", columns_to_drop)
print("Número de columnas tras la eliminación:", df.shape[1])

### **2.2 - Manejo de duplicados**
Se identifican y eliminan registros duplicados para evitar sesgos en el análisis. Se revisan duplicados completos y duplicados parciales en base a características clave como `price`, `floor_area` y `bedrooms`.

In [ ]:
# Contar filas duplicadas considerando todas las columnas
duplicates_total = df.duplicated().sum()
print(f"Número de filas completamente duplicadas: {duplicates_total}")

In [ ]:
# Seleccionar columnas clave para identificar duplicados parciales excluyendo IDs o metadatos
#
# Para detectar duplicados, se consideran las columnas que describen las características estructurales de la vivienda,
# evitando aquellas que son únicas para cada anuncio o irrelevantes para el análisis.
#
# Columnas **NO INCLUIDAS** en la detección de duplicados y razones:
# - `web_id` y `url`: Son identificadores únicos de cada anuncio, por lo que no sirven para detectar duplicados.
# - `title`: Contiene información redundante sobre el tipo de vivienda y ubicación, pero en formato texto libre, lo que
#   lo hace inconsistente para la comparación.
# - `professional_name`: El nombre de la agencia o propietario no influye en si un anuncio es duplicado o no.
# - `last_update`: Fecha de actualización del anuncio con valores inconsistentes (por ejemplo, "2 months" vs. "5 November"),
#   lo que impide su uso para identificar duplicados.
# - `location`: Contiene el nombre de la calle y en algunos casos el número, pero ya disponemos de `district` y `subdistrict`
#   que son más estructurados. Además, las direcciones pueden estar escritas de forma diferente en anuncios duplicados.
#
# Se opta por considerar características clave de la vivienda como `price`, `bedrooms`, `floor_area`, `year_built`, etc.,
# ya que un mismo inmueble puede aparecer varias veces con ligeras variaciones en el título, la inmobiliaria o la fecha de publicación.

duplicate_columns = ['type', 'price', 'deposit', 'floor_built', 'floor_area',
                     'year_built', 'bedrooms', 'bathrooms', 'second_hand',
                     'lift', 'garage_included', 'furnished', 'equipped_kitchen',
                     'fitted_wardrobes', 'air_conditioning', 'terrace', 'balcony',
                     'storeroom', 'swimming_pool', 'garden_area', 'district', 'subdistrict',
                     'postalcode', 'lat', 'lng']

# Contar filas que tienen duplicación en estas columnas
duplicates_partial = df.duplicated(subset=duplicate_columns).sum()
print(f"Número de filas duplicadas considerando solo columnas clave: {duplicates_partial}")

In [ ]:
# Eliminar filas completamente duplicadas
df.drop_duplicates(inplace=True)
print(f"Dataset después de eliminar duplicados completos: {df.shape}")

In [ ]:
# Eliminar duplicados en base a columnas clave, manteniendo la primera aparición
df.drop_duplicates(subset=duplicate_columns, keep='first', inplace=True)
print(f"Dataset después de eliminar duplicados parciales: {df.shape}")

### **2.3 - Eliminación de columnas con alto porcentaje de valores nulos**
Se eliminan variables con un gran número de valores nulos que no pueden imputarse de forma fiable, como `year_built`, `orientation` y `postalcode`. Se justifica la retención de otras variables con nulos si son clave para el análisis.

In [ ]:
# Calcular el porcentaje de valores nulos en cada columna
null_percentage = df.isnull().mean() * 100

# Mostrar el porcentaje de nulos ordenado de mayor a menor
print("Porcentaje de valores nulos por columna:")
print(null_percentage.sort_values(ascending=False))

In [ ]:
#  floor_area (57%) → Muy relevante para el precio. Mantener y evaluar imputación aunque tenga un alto porcentaje de nulos
#  deposit (41%) → No es muy relevante para el precio. Pese a ello, mantener y evaluar imputación aunque tenga un alto porcentaje de nulos

# Definir las columnas a eliminar
columns_to_drop = [
    'year_built',      # Muchos nulos (68%). Aunque debería ser relevante para price (construcciones modernas podrían ser más caras) es difícil de imputar correctamente
    'orientation',     # Muchos nulos (52%), difícil de imputar correctamente
    'postalcode'       # Porcentaje de nulos moderadamente alto (25%). Redundante con district y subdistrict, no aporta información adicional
]

# Eliminar las columnas del DataFrame
df.drop(columns=columns_to_drop, inplace=True)

# Mostrar las columnas eliminadas y el nuevo número de columnas
print("Columnas eliminadas por alto porcentaje de valores nulos:", columns_to_drop)
print("Número de columnas después de la eliminación:", df.shape[1])

### **2.4 - Saneamiento de valores en columnas categóricas**
Se homogeneizan valores en columnas categóricas como `type`, `district` y `subdistrict`, eliminando inconsistencias en mayúsculas/minúsculas, espacios en blanco y valores incorrectos.

In [ ]:
# Homogeneización de la columna 'type' para asegurar consistencia en los valores
print("Valores únicos en 'type' antes del saneamiento:", df['type'].unique(), "\n")

# Elimino espacios en blanco y capitalizo los valores
df['type'] = df['type'].str.strip().str.title()

print("Valores únicos en 'type' después del saneamiento:", df['type'].unique())

In [ ]:
# Limpieza a la columna 'floor' para estandarizar valores, corregir inconsistencias y pasar los datos a numérico mediante la función 'clear_floor_column'
df = clean_floor_column(df)

In [ ]:
# Homogeneización básica de 'district' y 'subdistrict'

# Estas columnas solo se utilizarán en el análisis exploratorio de datos (EDA),
# por lo que aplicamos una limpieza mínima para garantizar consistencia:
# - Convertimos los valores a minúsculas para evitar diferencias por mayúsculas/minúsculas.
# - Eliminamos espacios en blanco adicionales para garantizar uniformidad.
# - Reemplazamos valores 'nan' en formato string por valores NaN reales.
# - No realizamos agrupaciones de nombres similares, ya que no afectarán al modelo.

# Asegurar que sean de tipo 'object' antes de limpiar (evita problemas si son 'category')
df['district'] = df['district'].astype(str).str.strip().str.lower()
df['subdistrict'] = df['subdistrict'].astype(str).str.strip().str.lower()

# Reemplazar valores 'nan' string por NaN reales de manera robusta
df['district'] = df['district'].replace('nan', pd.NA).fillna(pd.NA)
df['subdistrict'] = df['subdistrict'].replace('nan', pd.NA).fillna(pd.NA)

# Mostramos los valores únicos después del saneamiento
print("Valores únicos en 'district' después del saneamiento:")
display(df['district'].unique())
print("\n")

print("Valores únicos en 'subdistrict' después del saneamiento:")
display(df['subdistrict'].unique())

### **2.5 - Saneamiento de valores en columnas numéricas**
Se sustituyen por `NaN` los valores negativos en columnas donde no pueden existir (e.g., `price`, `floor_area`, `bathrooms`). Además, se eliminan valores 0 en columnas donde no tienen sentido (`price`, `floor_area`, `floor_built`).

In [ ]:
# Convierto a NaN los valores negativos donde no pueden existir
not_neg_columns = ['price', 'deposit', 'floor_area', 'floor_built', 'floor', 'bedrooms', 'bathrooms']
for col in not_neg_columns:
    df.loc[df[col] < 0, col] = pd.NA

In [ ]:
# Convierto a NaN los valores 0 en columnas donde no tiene sentido
not_zero_columns = ['price', 'floor_area', 'floor_built']
for col in not_zero_columns:
    df.loc[df[col] == 0, col] = pd.NA

In [ ]:
# Mostrar valores negativos restantes
for col in not_neg_columns:
    neg_count = (df[col] < 0).sum()
    print(f" {neg_count} valores negativos restantes en '{col}'.")

In [ ]:
# Mostrar valores 0 restantes en las columnas afectadas
print("\nValores 0 restantes en columnas afectadas:")
for col in not_zero_columns:
    zero_count = (df[col] == 0).sum()
    print(f" - {col}: {zero_count} valores con valor 0")

In [ ]:
# Mostrar cuántos valores nulos quedaron en cada columna numérica saneada
print("\nValores nulos después del saneamiento:")
print(df[not_neg_columns].isna().sum())

### **2.6 - Saneamiento de valores de geolocalización**

El dataset incluye coordenadas de latitud (`lat`) y longitud (`lng`) obtenidas mediante la API de Google Maps. Sin embargo, algunos inmuebles pueden tener ubicaciones incorrectas fuera de la **Comunidad de Madrid**, posiblemente debido a errores en la geocodificación.

Para corregir esto, se define una función que calcula la distancia entre cada propiedad y el centro de Madrid. Se establece un **radio máximo de 60 km**, eliminando aquellas viviendas que se encuentren fuera de este límite.

Este proceso garantiza que los datos reflejen únicamente inmuebles dentro del área de interés y mejora la precisión del análisis.

In [ ]:
# Filtrar los registros fuera de Madrid
df["valid_location"] = df.apply(lambda row: is_within_madrid(row["lat"], row["lng"]), axis=1)

In [ ]:
# Mostrar cuántos registros serán eliminados
outliers_count = df[~df["valid_location"]].shape[0]
print(f"Número de registros fuera de la Comunidad de Madrid a eliminar: {outliers_count}")

In [ ]:
# Eliminar los registros fuera de Madrid
df = df[df["valid_location"]].drop(columns=["valid_location"]).reset_index(drop=True)

In [ ]:
# Confirmar el nuevo tamaño del dataset
print(f"Dataset después de eliminar ubicaciones incorrectas: {df.shape}")

### **2.7 - Conversión y optimización de tipos de datos**
Se ajustan los tipos de datos para reducir el consumo de memoria:
- **Enteros:** Se usan los tipos más eficientes (`Int8`, `Int16`, `Int32`).
- **Flotantes:** Se mantiene `float64` solo en coordenadas (`lat`, `lng`).
- **Booleanos:** Se convierten variables binarias a `bool`.
- **Categóricas:** `type`, `district` y `subdistrict` se transforman a `category`.

In [ ]:
# Se ajustan los tipos de datos para reducir el consumo de memoria:
# - Columnas enteras: Se asigna el tipo más eficiente según el rango de valores observados.
# - Columnas flotantes: Se mantienen en float64 solo las coordenadas, ya que requieren precisión.
# - Columnas booleanas: Se convierten a bool para representar valores binarios.
# - Columnas categóricas: Se transforman a 'category' para optimizar análisis y almacenamiento.

# Columnas enteras
df['price'] = df['price'].astype('Int32')
df['floor_built'] = df['floor_built'].astype('Int16')
df['floor'] = df['floor'].astype('Int8')
df['bedrooms'] = df['bedrooms'].astype('Int8')
df['bathrooms'] = df['bathrooms'].astype('Int8')
df['deposit'] = df['deposit'].astype('Int8')

# Columnas flotantes
df['floor_area'] = df['floor_area'].astype('float32')
df['lat'] = df['lat'].astype('float64') # Mantengo la precisión alta para las coordenadas
df['lng'] = df['lng'].astype('float64')

# Columnas booleanas
bool_columns = ['second_hand', 'lift', 'garage_included', 'furnished', 'equipped_kitchen',
                'fitted_wardrobes', 'air_conditioning', 'terrace', 'balcony',
                'storeroom', 'swimming_pool', 'garden_area', 'private_owner']
for col in bool_columns:
    df[col] = df[col].astype('bool')

# Columnas categóricas
categorical_columns = ['type', 'district', 'subdistrict']
for col in categorical_columns:
    df[col] = df[col].astype('category')

# Reviso cambios
print("Tipos de datos después de la conversión:")
print(df.dtypes)

## **3 - Análisis Exploratorio de Datos (EDA)**

En esta sección se realiza un **análisis detallado de las características del dataset** mediante estadísticas descriptivas y visualizaciones gráficas. Se busca comprender la distribución de los datos, detectar valores atípicos y analizar relaciones entre variables clave.


### **3.1 - Análisis univariado de variables**
Se estudia la distribución de cada variable individualmente para entender su comportamiento:
- **Histogramas** y **boxplots** (`sns.histplot`, `sns.boxplot`) para analizar la forma y dispersión de las distribuciones.
- **Identificación de valores atípicos (outliers)** de forma visual.

In [ ]:
# # Lista de variables numéricas a analizar
# num_vars = ['price', 'deposit', 'floor_built', 'floor_area', 'floor', 'bedrooms', 'bathrooms']
#
# # Crear figuras para histogramas y boxplots
# fig, axes = plt.subplots(nrows=len(num_vars), ncols=2, figsize=(12, len(num_vars) * 4))
#
# # Generar gráficos para cada variable numérica
# for i, var in enumerate(num_vars):
#     # Histograma con KDE
#     sns.histplot(df[var], bins=30, kde=True, ax=axes[i, 0], color='steelblue')
#     axes[i, 0].set_title(f'Histograma de {var}')
#
#     # Boxplot
#     sns.boxplot(x=df[var], ax=axes[i, 1], color='coral')
#     axes[i, 1].set_title(f'Boxplot de {var}')
#
# plt.tight_layout()
# plt.show()

### **3.2 - Análisis bivariado y relaciones entre variables**
Se exploran correlaciones entre variables para detectar patrones y posibles predictores:
- **Diagramas de dispersión** (`sns.scatterplot`) entre `price` y variables relevantes (`floor_area`, `bathrooms`, `bedrooms`).
- **Matriz de correlación** (`sns.heatmap`) para visualizar relaciones numéricas entre las variables.

In [ ]:
# # Matriz de correlación
# plt.figure(figsize=(14, 10))
# corr_matrix = df.corr(numeric_only=True)
#
# sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, annot_kws={"size": 8})
# plt.xticks(rotation=45, ha="right")
# plt.yticks(rotation=0)
# plt.title("Matriz de Correlación entre Variables Numéricas", fontsize=14)
# plt.show()

In [ ]:
# # Variables numéricas más correlacionadas con price
# key_vars = ['floor_area', 'bedrooms', 'bathrooms', 'floor_built']
#
# fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
#
# for i, var in enumerate(key_vars):
#     row, col = divmod(i, 2)
#     sns.scatterplot(data=df, x=var, y='price', alpha=0.5, ax=axes[row, col], color='royalblue')
#     sns.regplot(data=df, x=var, y='price', scatter=False, ax=axes[row, col], color='red', line_kws={"alpha":0.7})
#     axes[row, col].set_title(f'Relación entre Price y {var}')
#
# plt.tight_layout()
# plt.show()

In [ ]:
# # Pairplot con las principales variables
# sns.pairplot(df, vars=['price', 'floor_area', 'bedrooms', 'bathrooms', 'floor_built'], diag_kind="kde", corner=True)
# plt.show()

### **3.3 - Análisis multivariante y relaciones complejas**


### **3.4 - Mapa interactivo con coordenadas de viviendas**
Se genera una visualización geográfica de la distribución de viviendas en Madrid:
- Uso de **Folium** para trazar ubicaciones en un mapa.
- Aplicación de **MarkerCluster** para agrupar zonas con alta densidad de viviendas.

In [ ]:
# TODO Crear la nueva columna con la distancia al centro de Madrid. Eliminar valores fuera de Madrid

# Coordenadas del centro de Madrid
madrid_center = [40.4168, -3.7038]

# Crear un mapa base con zoom adecuado
m_cluster = folium.Map(location=madrid_center, zoom_start=12, tiles="cartodbpositron")

# Crear un objeto de clustering para agrupar los puntos
marker_cluster = MarkerCluster().add_to(m_cluster)

# Agregar marcadores agrupados con información relevante
for idx, row in df.iterrows():
    if pd.notna(row["lat"]) and pd.notna(row["lng"]):  # Asegurar que las coordenadas no sean nulas
        folium.Marker(
            location=[row["lat"], row["lng"]],
            popup=(
                f"<b>Precio:</b> {row['price']}€<br>"
                f"<b>Superficie:</b> {row['floor_area']} m²<br>"
                f"<b>Habitaciones:</b> {row['bedrooms']} | Baños: {row['bathrooms']}<br>"
                f"<b>Distrito:</b> {row['district'].capitalize() if pd.notna(row['district']) else 'Desconocido'}<br>"
                f"<b>Subdistrito:</b> {row['subdistrict'].capitalize() if pd.notna(row['subdistrict']) else 'Desconocido'}"
            ),
            icon=folium.Icon(color="blue", icon="home"),
        ).add_to(marker_cluster)

# Mostrar el mapa interactivo con clustering
m_cluster

## 4 - Preparación de los datos para el modelado

### 4.1 - Codificación de variables categóricas

### 4.2 - Escalado y normalización de variables numéricas

### 4.3 - Imputación de valores nulos

#### 4.3.1 - Análisis de valores nulos por columna

In [ ]:
# Filtrar columnas con valores nulos mayores a 0
null_values = df.isnull().sum()
null_values = null_values[null_values > 0]

# Mostrar resultados
print("Número de valores nulos por columna:")
print(null_values)

In [ ]:
# Filtrar columnas con porcentaje de nulos mayor a 0
null_percentage = df.isnull().mean() * 100
null_percentage = null_percentage[null_percentage > 0]

# Mostrar resultados
print("Porcentaje de valores nulos por columna:")
print(null_percentage)

#### 4.3.2 - deposit

In [ ]:
#### 2.7.2 - deposit
# Cálculo de estadísticas descriptivas de 'deposit'
# Se analiza tendencia central y dispersión para decidir la mejor estrategia de imputación.
print("Moda:", df['deposit'].mode()[0])
print("Mediana:", df['deposit'].median())
print("Media:", df['deposit'].mean())
print("Varianza:", df['deposit'].var())
print("Desviación estándar:", df['deposit'].std())
print("Rango:", df['deposit'].max() - df['deposit'].min())

In [ ]:
# Frecuencia absoluta
print(df['deposit'].value_counts())

# Frecuencia en porcentaje
print(df['deposit'].value_counts(normalize=True) * 100)

In [ ]:
# Ver correlaciones
print(df.select_dtypes(include=['number']).corr()['deposit'].sort_values(ascending=False))

In [ ]:
# Análisis de la dispersión de 'deposit' mediante percentiles e IQR
# Esto ayuda a entender su distribución y detectar posibles valores atípicos.
print("Percentil 25:", df['deposit'].quantile(0.25))
print("Percentil 50 (Mediana):", df['deposit'].quantile(0.50))
print("Percentil 75:", df['deposit'].quantile(0.75))
print("IQR:", df['deposit'].quantile(0.75) - df['deposit'].quantile(0.25))

In [ ]:
# Imputación de valores nulos en 'deposit'
#
# La mayoría de los valores en 'deposit' son 1 (59% de los casos), seguido de 2 (35%).
# Dado que la media (1.47) está sesgada por unos pocos valores más altos y la mediana también es 1,
# se decide imputar los valores nulos con la moda (1), ya que refleja mejor la tendencia real de los datos.
df['deposit'] = df['deposit'].fillna(1)

#### 4.3.3 - floor_build

In [ ]:
# Dado que los valores nulos representan solo el 0.1% de la columna, se imputan con la mediana para mantener la distribución sin sesgos y evitar la influencia de valores atípicos.
median_floor_built = df['floor_built'].median()
df['floor_built'] = df['floor_built'].fillna(median_floor_built)

#### 4.3.4 - floor_area

In [ ]:
# correlación con price (0.72), bedrooms (0.51), bathrooms (0.69) y floor_built (0.70)

# Crear figura con 2 filas y 3 columnas
fig, axes = plt.subplots(2, 3, figsize=(18, 10))  # Se ajustó el tamaño para mejor proporción

# Gráfico de distribución de floor_area
sns.histplot(df['floor_area'].dropna(), bins=30, kde=True, ax=axes[0, 0])
axes[0, 0].set_title("Distribución de Floor Area")

# Scatterplots con línea de tendencia
sns.regplot(x=df['price'], y=df['floor_area'], ax=axes[0, 1], scatter_kws={'alpha':0.5}, line_kws={"color": "red"})
axes[0, 1].set_title("Floor Area vs Price")

sns.regplot(x=df['bedrooms'], y=df['floor_area'], ax=axes[0, 2], scatter_kws={'alpha':0.5}, line_kws={"color": "red"})
axes[0, 2].set_title("Floor Area vs Bedrooms")

sns.regplot(x=df['bathrooms'], y=df['floor_area'], ax=axes[1, 0], scatter_kws={'alpha':0.5}, line_kws={"color": "red"})
axes[1, 0].set_title("Floor Area vs Bathrooms")

sns.regplot(x=df['floor_built'], y=df['floor_area'], ax=axes[1, 1], scatter_kws={'alpha':0.5}, line_kws={"color": "red"})
axes[1, 1].set_title("Floor Area vs Floor Built")

# Matriz de correlación
corr_matrix = df[['floor_area', 'price', 'bedrooms', 'bathrooms', 'floor_built']].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, ax=axes[1, 2])
axes[1, 2].set_title("Matriz de Correlación")

# Ajustar espaciado entre gráficos
plt.tight_layout()
plt.show()

In [ ]:
# Se elige la Regresión Lineal Múltiple para imputar floor_area porque presenta una fuerte correlación con price (0.73),
# bedrooms (0.74), bathrooms (0.79) y floor_built (0.92).
# Las gráficas muestran una relación lineal clara entre floor_area y estas variables, lo que indica que pueden predecirlo de manera precisa.
# Además, la distribución de floor_area está sesgada a la derecha, por lo que métodos como la mediana podrían subestimar los valores más altos.

# 1Selección de las Variables Predictoras y Variable Dependiente
predictors = ['price', 'bedrooms', 'bathrooms', 'floor_built']
target = 'floor_area'

# División del Dataset
df_train = df.dropna(subset=[target])  # Filas sin nulos en floor_area (para entrenar)
df_missing = df[df[target].isna()]  # Filas con nulos en floor_area (para predecir)

# Preparación de los Datos para el Modelo
X_train = df_train[predictors]
y_train = df_train[target]

# Entrenamiento del Modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluación del Modelo
y_pred_train = model.predict(X_train)
r2 = r2_score(y_train, y_pred_train)
mae = mean_absolute_error(y_train, y_pred_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
mape = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100  # Error absoluto porcentual medio

print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")  # Muestra el error en porcentaje

# 6️⃣ Predicción e Imputación de Valores Nulos
if not df_missing.empty:
    X_missing = df_missing[predictors]
    df.loc[df[target].isna(), target] = model.predict(X_missing).astype('float32')

In [ ]:
#  Evaluación del Modelo de Imputación para `floor_area`
# - R² = 0.872 → El modelo explica el 87.2% de la variabilidad en `floor_area`, lo que indica un buen ajuste.
# - MAE = 9.47 → En promedio, el error absoluto en la imputación es de 9.47 m², un error moderado en la escala de la variable.
# - RMSE = 26.14 → El error cuadrático medio sugiere que existen algunos valores más alejados de la realidad (posiblemente por outliers).
# - MAPE = 10.21% → En promedio, los valores imputados tienen un error del 10.21% respecto a los valores reales,
#   lo que indica una imputación razonablemente precisa, aunque con margen de mejora en valores extremos.
#
#   Conclusión: La regresión lineal múltiple ofrece una buena precisión en la imputación de `floor_area`, con un R² alto y errores aceptables.
#   Sin embargo, los valores más altos de `floor_area` podrían estar generando desviaciones en la predicción.
#   Se recomienda revisar outliers y considerar técnicas más avanzadas (como regresión robusta o árboles de decisión) para mejorar la estimación.


#### 4.3.5 - floor


## 5 - Modelado

## 6 - Evaluación y conclusiones